In [3]:
import random, json, pickle, numpy as np, nltk

from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

# Initialiser le lemmatiseur
lemmatizer = WordNetLemmatizer()

# Charger les nouvelles données intents.json
with open('../intents.json') as file:
    intents = json.load(file)

# Initialisation des listes
words = []
classes = []
documents = []
ignore_words = ['?', '!', '.', ',']

# Boucle pour extraire les mots et classes depuis intents.json
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokeniser chaque mot
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        # Ajouter le pattern et l'intent (catégorie)
        documents.append((word_list, intent['tag']))
        # Ajouter à la liste des classes si ce n'est pas déjà fait
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatisation et nettoyage des mots
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(set(words))

# Trier les classes
classes = sorted(set(classes))

# Sauvegarder les nouveaux fichiers words.pkl et classes.pkl
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Création de l'entraînement des données
training = []
output_empty = [0] * len(classes)

# Créer le bag of words pour chaque pattern
for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for w in words:
        bag.append(1) if w in word_patterns else bag.append(0)
    
    # Marque pour la classe correspondante
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# Shuffle et conversion des données
random.shuffle(training)

# Conversion en format de tableau NumPy pour l'entraînement
train_x = np.array([x[0] for x in training])
train_y = np.array([x[1] for x in training])

# Réentraîner le modèle
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compilation du modèle
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Entraînement du modèle
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

# Sauvegarder le nouveau modèle
model.save('chatbot_model.h5')

print("Réentraînement terminé et modèle sauvegardé.")

Epoch 1/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.1579 - loss: 2.4064
Epoch 2/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.1471 - loss: 2.4017   
Epoch 3/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2667 - loss: 2.3116
Epoch 4/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3918 - loss: 2.2136
Epoch 5/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2474 - loss: 2.0840
Epoch 6/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4838 - loss: 1.9764
Epoch 7/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4240 - loss: 1.8276  
Epoch 8/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3632 - loss: 1.7300
Epoch 9/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3525 - loss: 1.7775
Epoch 10/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6894 - loss: 1.4556
Epoch 11/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6795 - loss: 1.3619  
Epoch 12/200
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - a

Réentraînement terminé et modèle sauvegardé.
